These recommender systems are next generation recommendations systems, which fall into the hyper-personalization category. It's natural that there won't be an end to the needs of humans. The more we get, the more we want. Though content-based recommender systems are efficient, targeted at an individual level, and consider the user's personal preferences alone while building recommendation engines, people wanted recommendation engines to be more personalized. For example, a person going on a trip alone may need a book to read whereas the same person may need beer if he is travelling with friends. Similarly, the same person might require diapers, medicines, snacks, and so on if he is going with his own family. People at different places at different
times with different company have different needs. Our recommender systems should be robust enough to handle such scenarios. Such hyper personalized recommender systems, which cater to different recommendations to the same person based on his current context, are known as context-aware recommender systems.

<h2> Building a context-aware recommender systems</h2>
Building a context-aware recommender system is more like extending a content recommender system. Building a context-aware system typically involves adding the
context dimension on top of content recommenders, as shown in the following figure:
In the preceding figure, we can observe that context dimension is added on top of a contentbased recommendation engine model, and then recommendations are generated. As we
discussed in Chapter 3, Recommendation Engines Explained, there are two popular types of approaches for building context-aware recommendations:
Pre-filtering approaches
Post-filtering approaches
In this section, we will use post-filtering techniques to build context-aware recommender systems.

<h2>Context-aware recommendations using R</h2>
In the previous section, we built a content-based recommendation engine. In this section, we will extend the previous model to include context information and generate a contextaware recommendation engine.
The usual practice of building context-aware systems is to add a time dimension to the content-based recommendations.

Let's try building context aware systems using R. The steps for building context-aware systems in R are as follows:
1. Define context.
2. Create a context profile with respect to a user for item content.
3. Generate recommendations for a context

<h3>Defining the context</h3>
The first step is to define the context that we will be including in our recommendations. In
the previous section, we used the MovieLens dataset to build content-based
recommendation engines. In the dataset, we have a time component, timestamp, in the
rating data. We shall use this variable for our context-aware recommendation systems.
We will extend the R code we used while building content-based recommendations

In [ ]:
# We load the full MovieLens ratings dataset as follows:
raw_data = read.csv("C:/Suresh/R&D/packtPublications/reco_engines/drafts/personalRecos/udata.csv",sep="\t",header=F)
colnames(raw_data) = c("UserId","MovieId","Rating","TimeStamp")

# See the sample data using head() function:
head(raw_data)

# We load movies dataset:
movies = read.csv("C:/Suresh/R&D/packtPublications/reco_engines/drafts/personalRecos/uitem.csv",sep="|",header=F)

# Then we add column names to the movies data frame:
colnames(movies) = c("MovieId","MovieTitle","ReleaseDate","VideoReleaseDate","IMDbURL","Unknown","Action","Adventure","Animation","Children","Comedy","Crime","Documentary","Drama","Fantasy","FilmNoir","Horror","Musical","Mystery","Romance","SciFi","Thriller","War","Western")

# Next, we remove unwanted columns from the data frame:
movies = movies[,-c(2:5)]

# We merge the Movies and Ratings datasets using merge() function
ratings_ctx = merge(x = raw_data, y = movies, by = "MovieId", all.x = TRUE)


The context that we want to introduce to our previous content-based recommendation is the hour of the day, that is, our recommendations will be made as per the time of the day. The set of recommendations for an active user will be different for each hour of the day. Usually, these changes in recommendations are due to the ordering of the recommendations as per the hour. We will see next how we achieve this.

<h3>Creating context profile</h3>
In the following section, we shall write code to create context profile of the user. We chose the timestamp information available in the dataset and calculate the preference value for movie genres for each user for each hour of the day. This context profile information is used for generating context aware recommendations.

In [ ]:
# We extract timestamp from the ratings dataset:
ts = ratings_ctx$TimeStamp

# Then, we convert it into a POSIXlt date object and using hour property to extract hour of the day:
hours <- as.POSIXlt(ts,origin="1960-10-01")$hour

# See below for sample data:
head(ts)
head(hours)

# We can append the hours back on to the ratings dataset:
ratings_ctx = data.frame(cbind(ratings_ctx,hours))

# Now, let's start building a context profile for a user with the user ID 943:

# Extract ratings information for the active user(943) and removing UserId, MovieId, Rating, Timestamp columns, as shown as follow:
UCP = ratings_ctx[(ratings_ctx$UserId == 943),][,-c(2,3,4,5)]

# As a next step, we compute the columns of all the item features. This columnwise sum is used to compute the preferences for the item features for each hour of the day.

# We compute the column wide sum of each column using aggregate() function:
UCP_pref = aggregate(.~hours,UCP[,-1],sum)

# From the preceding figure, we can see the time preferences for each of the movie genres for the active user 943. We can observe that during the ninth hour of the day, the user watches more movies, especially action/drama/comedy movies:

# We can normalize the preceding data between 0-1 using following function:
UCP_pref_sc = cbind(context = UCP_pref[,1],t(apply(UCP_pref[,-1], 1, function(x)(x-min(x))/(max(x)-min(x)))))


<h3>Generating context-aware recommendations</h3>
Now that we have created the context profile for the active user, let's start generating context-aware recommendations for the user.
For this, we shall reuse the recommend object built using R, which contains content recommendations for all the users.


In [ ]:
# Let's see the recommendations made to the user 943 using the content-based system:
recommend$MovieId

# Now, to these content recommendations, we add our time or hour of the day dimension and then generate recommendations as per the current context.

# We merge recommendations and movies dataset using merge() function:
UCP_pref_content = merge(x = recommend, y = movies, by = "MovieId", all.x = TRUE)


With the preceding step, we have computed all the required matrices, user context profile (UCP_Pref_SC) and user content recommendations (UCP_Pref_content).

Suppose we want to generate recommendations for the user at the ninth hour; we just need to perform an element wise multiplication of user content recommendations and the context row for the ninth hour of the day from the UCP_pref_SC object. This is given as follows:



In [ ]:
# Performing element wise multiplication for the User content recommendations and the ninth hour context preferences for the user:
active_user =cbind(UCP_pref_content$MovieId,(as.matrix(UCP_pref_content[,-c(1,2,3)]) %*% as.matrix(UCP_pref_sc[4,2:19])))

# The results can be seen as follows; we can observe that the preference for MovieId 3 is 0.5 where as for MovieId 4 the preference is 2.8
head(active_user)

# We can create a dataframe object of the prediction object:
active_user_df = as.data.frame(active_user)

# Next, we add column names to the predictions object:
names(active_user_df) = c('MovieId','SimVal')

# Then we sort the results:
FinalPredicitons_943 = active_user_df[order(-active_user_df$SimVal),]
